In [1]:
import requests


In [7]:
# retrieve download link
result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
    "company": "Alasco",
    "company_id": "HRB 205171",
    "documents": ["gs", "si"]
})

In [17]:
response = result.json()

# filter python list for document url with type gs
gs_doc_url = list(filter(lambda x: x["type"] == "gs", response["document_urls"]))[0]["url"]

# download document
gs_doc = requests.get(gs_doc_url)

# store document as pdf file in folder output/
with open(f"output/gs-{response['name']}.pdf", "wb") as f:
    f.write(gs_doc.content)

In [20]:
# load csv list into list of dictionaries
import csv

with open('startupdetector_founders_v1_MF - Startups.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    startup_list = list(reader)

print(startup_list[0])

startup_list[0]["Company_Name"]
startup_list[0]["Aktenzeichen"]



{'#': '1', 'Company_Name': 'BCN Food Tech GmbH', 'Aktenzeichen': 'HRB 272187', 'Date_Incorporation': '2021-12-31', 'CMP_Status': 'A', 'Amtsgericht': 'München', 'Street': 'Pestalozzistraße 34', 'Postcode': 'D-80469', 'City': 'München', 'Industry': 'Lebensmittel', 'WZ2008': '10', 'Startup_Website': '–', 'LinkedIn_URL': '–', 'Crunchbase_URL': '–', 'Northdata_URL': 'https://www.northdata.com/BCN+Food+Tech+GmbH,+M%C3%BCnchen/HRB+272187', 'Comment': '', 'Founding_Team_Size': '1'}


In [ ]:
# retrieve download link
result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
    "company": startup_list[0]["Company_Name"],
    "company_id": startup_list[0]["Aktenzeichen"],
    "documents": ["gs", "si"]
})

response = result.json()

# filter python list for document url with type gs
gs_doc_url = list(filter(lambda x: x["type"] == "gs", response["document_urls"]))[0]["url"]

# download document
gs_doc = requests.get(gs_doc_url)

# store document as pdf file in folder output/
with open(f"output/gs-{response['name']}.pdf", "wb") as f:
    f.write(gs_doc.content)